# DATASCI W261: Machine Learning at Scale 

**Name: Carlos Eduardo Rodriguez Castillo**

**email: cerodriguez@berkeley.edu**

**Week 4**

**Section 2**

#### HW4.0

What is MrJob? How is it different to Hadoop MapReduce?

##### ANSWER:

MrJob is a python framework for interacting with the Hadoop MapReduce programming infrastructure.

It is different from Hadoop MapReduce programming framework in that it is a purely based framework (Hadoop MapReduce supports scripts in several languages). 

Additionally, MrJob allows the coder to run MapReduce jobs locally (without Hadoop) while Hadoop MapReduce by definition requires a running hadoop infrastructure to function.

MrJob also abstracts the coder away from manual tasks that are required in the Hadoop MapReduce framework such as data serialization and deserialization between tasks.

Finally, MrJob has a native integration with Amazon Elastic MapReduce which permits us to easily run large scale jobs on a cluster in the AWS cloud.

What are the mapper_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?

##### ANSWER:

The methods can be described as follows:

- __mapper_init__: a function with the initialization code for each of the mapper nodes. The function defines an action to take before the mapper processes any input.
- __mapper_final()__: a function with the termination code for each of the mapper nodes. The function defines an action to be taken by the mapper after it gets to the end of its input.
- __combiner_final()__: a function with the termination code for each of the combiners. The function defines an action to be taken by the combiner after it gets to the end of its input.
- __reduce_final()__: a function with the termination code for each of the reducers. The function defines an action to be taken by the reducer after it gets to the end of its input.

### HW 4.0.1 Total sort using 1 and 3 reducers (a repeat of HW2.1) [OPTIONAL]

WARNING: per-step jobconf has bugs that affect Total sorts/partitions etc.
For MRJob,  Sort, partition code via the MRJob config does NOT work in local mode (known bug/feature which I believe has not been fixed as of June 2016). 
So you will need to run in the cloud (e.g.  in AWS).
It's issue #616 in github:  "Inline and Local modes should support per-step jobconf #616".  https://github.com/Yelp/mrjob/issues/616
To overcome this issue run your MRJob jobs on the cloud using -r hadoop or -r emr:

       #!python MostFrequentVisits.py -r hadoop anonymous-msweb_converted.data

NOTE:  
  Hadoop will always give a total sort on the key (i.e., key part of the key-value pairs produced by the mappers) when using just one reducer.
  When using multiple reducers Hadoop will by default give you a partial sort (i.e., all records within a partition will be sorted by the key (i.e., key part of the key-value pairs produced by the mappers) .
  To achieve a total sort one needs to write a customer mapper to to prepend a partition key to each record,  and then do a secondary sort or the resulting records (This can be done with two map-reduce jobs or combined into one only job)


[next up: TOTAL SORT using a single reducer]
Given as input: Records of the form <integer, “NA”>, where integer is any integer, and “NA” is just the empty string.
Output: sorted key value pairs of the form <integer, “NA”> in decreasing order using a single reducer. 
Write code to generate N  random records of the form <integer, “NA”>. Let N = 10,000.
Write the python Hadoop streaming map-reduce job to perform this sort. Display the top 10 biggest numbers. Display the 10 smallest numbers.

[next up: TOTAL SORT using multiple reducers] 
What happens if you have multiple reducers? Do you need additional steps? Explain. Feel free to code this up (This is an optional task). 
Write code to generate N  random records of the form <integer, “NA”>. Let N = 10,000.
Write the python Hadoop streaming map-reduce job to perform this sort using 3 reducers. Display the top 10 biggest numbers. Display the 10 smallest numbers.

HINT: you might need a jobconf for the sort step in your overall MRJob Script      
        JOBCONF_STEP2 = {
            'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'stream.num.map.output.key.field': 2,
            'stream.map.output.field.separator':',',    
            'mapreduce.partition.keycomparator.options': '-k2,2nr -k1,1',
            'mapreduce.job.reduces': '1',
        }

HINT2: Use the proper separator to separate the key from the key-value pair generated by the mapper.

### HW4.1

What is serialization in the context of MrJob or Hadoop?

__ANSWER__:

In the context of MrJob or Hadoop, serialization is the process of turning data that is passed from one task to another into a format for efficiently shuffling the data across the network. Data is also serialized for efficient storage of the data (be it in memory or disk).

Finally, the performance of MrJob/Hadoop is reliant on using an efficient serialization of the data being processed.

When is it used in these frameworks?

__ANSWER__:

Serialization is used in these frameworks when writing to disk and when passing data from one task to the next (that is from mapper to reducer, mapper to combiner, or combiner to reducer).

What is the default serialization mode for input and outputs for MrJob?

__ANSWER__:

The default serialization mode for input for MrJob is 'RawValueProtocol' which means that MrJob just reads in each line from the input as a string.

The default serialization mode for output for MrJob 'JSONProtocol' which means that MrJob writes as a final output JSON strings separated by a tab character.

### HW4.2 

Recall the Microsoft logfiles data from the async lecture. The logfiles are respectively described and located at:

https://kdd.ics.uci.edu/databases/msweb/msweb.html
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/

This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.

 Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:

C,"10001",10001   #Visitor id 10001

V,1000,1          #Visit by Visitor 10001 to page id 1000

V,1001,1          #Visit by Visitor 10001 to page id 1001

V,1002,1          #Visit by Visitor 10001 to page id 1002

C,"10002",10002   #Visitor id 10001

V
Note: #denotes comments
to the format:

V,1000,1,C, 10001

V,1001,1,C, 10001

V,1002,1,C, 10001

Write the python code to accomplish this.

##### ANSWER:

Please see python code below to accomplish this. First, I pull the data down:

In [ ]:
!cd /home/cloudera/w261/HW4
!mkdir -p data
!mkdir -p src
!cd data
!wget "http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/anonymous-msweb.data"
!mv anonymous-msweb.data data

Then I wrote the file to do the preprocessing:

In [13]:
%%writefile preprocess.py
#!/usr/bin/python
"""
Single node data preprocessing for HW4.2
"""
__author__ = "Carlos Eduardo Rodriguez Castillo"
__email__ = "cerodriguez@berkeley.edu"

import sys
import re

url_dict = {}

for line in sys.stdin:
    line = line.strip()
    elements = line.split(",")
    ## Given the design of the raw data file, 
    ## the script will first populate the url_dict
    if elements[0] == 'A':
        url_dict[elements[1]] = 'http://www.microsoft.com' + elements[4].strip('\"')
    elif elements[0] == 'C':
        visitor_data = elements[0] + ',' + elements[2]
        continue
    elif elements[0] == 'V':
        ## this is formatted as 'V,[URL_ID],[URL]
        visit_data = elements[0] + ',' + elements[1] + ',' + url_dict[elements[1]]
        ## this is formatted as 'V,[URL_ID],[URL],C,[USER_ID]
        processed_line = visit_data + ',' + visitor_data
        print processed_line

Overwriting preprocess.py


Finally, I tested the script on a small dataset:

In [14]:
!chmod a+x preprocess.py

In [15]:
!head -n 314 /home/cloudera/w261/HW4/data/anonymous-msweb.data > \
/home/cloudera/w261/HW4/data/mini-anonymous-msweb.data
!head -n 10 /home/cloudera/w261/HW4/data/anonymous-msweb.data

I,4,"www.microsoft.com","created by getlog.pl"
T,1,"VRoot",0,0,"VRoot"
N,0,"0"
N,1,"1"
T,2,"Hide1",0,0,"Hide"
N,0,"0"
N,1,"1"
A,1287,1,"International AutoRoute","/autoroute"
A,1288,1,"library","/library"
A,1289,1,"Master Chef Product Information","/masterchef"


In [16]:
!./preprocess.py < \
/home/cloudera/w261/HW4/data/mini-anonymous-msweb.data # testing the preprocessing phase

V,1000,http://www.microsoft.com/regwiz,C,10001
V,1001,http://www.microsoft.com/support,C,10001
V,1002,http://www.microsoft.com/athome,C,10001
V,1001,http://www.microsoft.com/support,C,10002
V,1003,http://www.microsoft.com/kb,C,10002
V,1001,http://www.microsoft.com/support,C,10003
V,1003,http://www.microsoft.com/kb,C,10003
V,1004,http://www.microsoft.com/search,C,10003
V,1005,http://www.microsoft.com/norge,C,10004


Seems to work, woohoo! Now, let's run the preprocessing script on the whole dataset:

In [17]:
!./preprocess.py < \
/home/cloudera/w261/HW4/data/anonymous-msweb.data \
> /home/cloudera/w261/HW4/data/anonymous-msweb-processed.data
!ls /home/cloudera/w261/HW4/data/

anonymous-msweb.data  anonymous-msweb-processed.data  mini-anonymous-msweb.data


In [18]:
!head /home/cloudera/w261/HW4/data/anonymous-msweb-processed.data

V,1000,http://www.microsoft.com/regwiz,C,10001
V,1001,http://www.microsoft.com/support,C,10001
V,1002,http://www.microsoft.com/athome,C,10001
V,1001,http://www.microsoft.com/support,C,10002
V,1003,http://www.microsoft.com/kb,C,10002
V,1001,http://www.microsoft.com/support,C,10003
V,1003,http://www.microsoft.com/kb,C,10003
V,1004,http://www.microsoft.com/search,C,10003
V,1005,http://www.microsoft.com/norge,C,10004
V,1006,http://www.microsoft.com/misc,C,10005


### HW4.3

Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).

WARNING: per-step jobconf has bugs that affect Total sorts/partitions etc.
For MRJob,  Sort, partition code via the MRJob config does NOT work in local mode (known bug/feature which I believe has not been fixed as of June 2016). 
So you will need to run in the cloud (e.g.  in AWS).
It's issue #616 in github:  "Inline and Local modes should support per-step jobconf #616".  https://github.com/Yelp/mrjob/issues/616
To overcome this issue run your MRJob jobs on the cloud using -r hadoop or -r emr:

       #!python MostFrequentVisits.py -r hadoop anonymous-msweb_converted.data


##### ANSWER:

The five most frequently visited pages (identified by page ID) are as follows:

1. Page ID: "1008"	Count: 10836
2. Page ID: "1034"	Count: 9383
3. Page ID: "1004"	Count: 8463
4. Page ID: "1018"	Count: 5330
5. Page ID: "1017"	Count: 5108

Below is the MrJob I used to come to this solution:

In [87]:
%%writefile MostFrequentlyVisited.py
#!/home/cloudera/anaconda2/bin/python

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys

class MRMostFrequentlyVisited(MRJob):
    
    COUNTER = 0
    def mapper_count_visits(self, _, line):
        line = line.strip()
        record_elements = line.split(",")
        page = record_elements[1]
        yield (page, 1)
            
    def reducer_sum_visits(self, page, counts):
        yield (page, sum(counts))
    
    def reducer_sort_sumVisits(self, page, counts):
        yield (page, sum(counts))
    
    def steps(self):
        JOB_CONF_STEP2 = {
            'mapreduce.job.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'stream.num.map.output.key.field': 2,
            'stream.map.output.field.separator':',',
            'mapreduce.partition.keycomparator.options': "-k2,2nr -k1,1", # sort pages by count and then by ID
             'mapreduce.job.reduces': 1
        }
        return [
            MRStep(mapper=self.mapper_count_visits,
                  reducer=self.reducer_sum_visits),
            MRStep(jobconf=JOB_CONF_STEP2,
                   reducer=self.reducer_sort_sumVisits)
        ]
    
if __name__ == '__main__':
    MRMostFrequentlyVisited().run()

Overwriting MostFrequentlyVisited.py


In [88]:
!chmod a+x MostFrequentlyVisited.py

In [89]:
!./MostFrequentlyVisited.py -r hadoop \
/home/cloudera/w261/HW4/data/anonymous-msweb-processed.data \
| head -n 5

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MostFrequentlyVisited.cloudera.20160610.225613.275472
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Copying local files to hdfs:///user/cloudera/tmp/mrjob/MostFrequentlyVisited.cloudera.20160610.225613.275472/files/...
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Running step 1 of 2...
  packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.7.0.jar] /tmp/streamjob5487563921880336629.jar tmpDir=null
  Connecting to ResourceManager at quickstart.cloudera/127.0.0.1:8032
  Connecting to ResourceManager at quickstart.cloudera/127.0.0.1:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1465484910625_0030
  Submitted application application_1465484910625_0030

### HW4.4

Find the most frequent visitor of each page using MrJob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.

##### ANSWER:

Using the two MapReduce jobs defined below we find the most frequent visitor of each of the pages and we deposit the answers in the _hw4_4_output file_. __By exploring the file we find that the maximum number of visits that any given user in the dataset makes to any given page is one!__

In [64]:
%%writefile MostFrequentVisitorPerPageMRJob.py
#!/home/cloudera/anaconda2/bin/python

from mrjob.job import MRJob
from mrjob.step import MRStep
import sys

class MostFrequentVisitorPerPageMRJob(MRJob):
    
    COUNTER = 0
    def mapper_count_visits(self, _, line):
        line = line.strip()
        record_elements = line.split(",")
        page_id = record_elements[1]
        page_url = record_elements[2]
        user_id = record_elements[4]
        key = page_id + ',' + page_url + ',' + user_id
        yield (key, 1)
            
    def reducer_sum_visits(self, key, counts):
        yield (key, sum(counts))
    
    def reducer_sort_sumVisits(self, key, counts):
        yield (key, max(counts))
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper_count_visits,
                  reducer=self.reducer_sum_visits),
            MRStep(reducer=self.reducer_sort_sumVisits)
        ]
    
if __name__ == '__main__':
    MostFrequentVisitorPerPageMRJob().run()

Overwriting MostFrequentVisitorPerPageMRJob.py


In [65]:
!chmod a+x MostFrequentVisitorPerPageMRJob.py
!./MostFrequentVisitorPerPageMRJob.py -r hadoop \
/home/cloudera/w261/HW4/data/anonymous-msweb-processed.data \
> hw4_4_output

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MostFrequentVisitorPerPageMRJob.cloudera.20160610.035008.835353
Looking for hadoop binary in $PATH...
Found hadoop binary: /usr/bin/hadoop
Using Hadoop version 2.6.0
Copying local files to hdfs:///user/cloudera/tmp/mrjob/MostFrequentVisitorPerPageMRJob.cloudera.20160610.035008.835353/files/...
Looking for Hadoop streaming jar in /home/hadoop/contrib...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Running step 1 of 2...
  packageJobJar: [] [/usr/jars/hadoop-streaming-2.6.0-cdh5.7.0.jar] /tmp/streamjob3207474619294881084.jar tmpDir=null
  Connecting to ResourceManager at quickstart.cloudera/127.0.0.1:8032
  Connecting to ResourceManager at quickstart.cloudera/127.0.0.1:8032
  Total input paths to process : 1
  number of splits:2
  Submitting tokens for job: job_1465484910625_0028
  Submitted application applicatio

In [66]:
!head hw4_4_output

"1000,http://www.microsoft.com/regwiz,10001"	1
"1000,http://www.microsoft.com/regwiz,10010"	1
"1000,http://www.microsoft.com/regwiz,10039"	1
"1000,http://www.microsoft.com/regwiz,10073"	1
"1000,http://www.microsoft.com/regwiz,10087"	1
"1000,http://www.microsoft.com/regwiz,10101"	1
"1000,http://www.microsoft.com/regwiz,10132"	1
"1000,http://www.microsoft.com/regwiz,10141"	1
"1000,http://www.microsoft.com/regwiz,10154"	1
"1000,http://www.microsoft.com/regwiz,10162"	1


### HW4.5

HW 4.5 Clustering Tweet Dataset

Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of  recent research,
which spawned this dataset:

http://arxiv.org/abs/1505.04342
http://arxiv.org/abs/1508.01843

The main data lie in the accompanying file:

topUsers_Apr-Jul_2014_1000-words.txt

and are of the form:

USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
.
.

where

USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm in MrJob on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words (generate 1000 random numbers and normalize the vectors)

(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 

(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 

(D) K=4 "trained" centroids, determined by the sums across the classes. Use use the (row-normalized) class-level aggregates as 'trained' starting centroids (i.e., the training is already done for you!). Note that you do not have to compute the aggregated distribution or the  class-aggregated distributions, which are rows in the auxiliary file:

topUsers_Apr-Jul_2014_1000-words_summaries.txt

Row 1: Words

Row 2: Aggregated distribution across all classes

Row 3-6 class-aggregated distributions for clases 0-3

For (A),  we select 4 users randomly from a uniform distribution [1,...,1,000]

For (B), (C), and (D)  you will have to use data from the auxiliary file: 

topUsers_Apr-Jul_2014_1000-words_summaries.txt

This file contains 5 special word-frequency distributions:

(1) The 1000-user-wide aggregate, which you will perturb for initializations in parts (B) and (C), and (2-5) The 4 class-level aggregates for each of the user-type classes (0/1/2/3)


In parts (B) and (C), you will have to perturb the 1000-user aggregate (after initially normalizing by its sum, which is also provided). So if in (B) you want to create 2 perturbations of the aggregate, start with (1), normalize, and generate 1000 random numbers uniformly from the unit interval (0,1) twice (for two centroids), using:

from numpy import random
numbers = random.sample(1000)

Take these 1000 numbers and add them (component-wise) to the 1000-user aggregate, and then renormalize to obtain one of your aggregate-perturbed initial centroids.


###################################################################################
\## Geneate random initial centroids around the global aggregate

\## Part (B) and (C) of this question
###################################################################################

def startCentroidsBC(k):

    counter = 0
    for line in open("topUsers_Apr-Jul_2014_1000-words_summaries.txt").readlines():
        if counter == 2:        
            data = re.split(",",line)
            globalAggregate = [float(data[i+3])/float(data[2]) for i in range(1000)]
        counter += 1
    \## perturb the global aggregate for the four initializations    
    centroids = []
    for i in range(k):
        rndpoints = random.sample(1000)
        peturpoints = [rndpoints[n]/10+globalAggregate[n] for n in range(1000)]
        centroids.append(peturpoints)
        total = 0
        for j in range(len(centroids[i])):
            total += centroids[i][j]
        for j in range(len(centroids[i])):
            centroids[i][j] = centroids[i][j]/total
    return centroids



——
For experiments A, B, C and D and iterate until a threshold (try 0.001) is reached. After convergence, print out a summary of the classes present in each cluster. In particular, report the composition as measured by the total portion of each class type (0-3) contained in each cluster, and discuss your findings and any differences in outcomes across parts A-D.

In [67]:
!wget "https://www.dropbox.com/sh/m0nxsf4vs5cyrp2/AAD5G0PHKMgdqTPC1w-2rR2ya/topUsers_Apr-Jul_2014_1000-words_summaries.txt?dl=0"

--2016-06-10 13:30:07--  https://www.dropbox.com/sh/m0nxsf4vs5cyrp2/AAD5G0PHKMgdqTPC1w-2rR2ya/topUsers_Apr-Jul_2014_1000-words_summaries.txt?dl=0
Resolving www.dropbox.com... 162.125.4.1
Connecting to www.dropbox.com|162.125.4.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/nb7DGyGrJ1b8Rky83ThFf5eJZOp5n3tREARyL5eQmJjKmwK3gYyAvtxLrAqulshe/file [following]
--2016-06-10 13:30:08--  https://dl.dropboxusercontent.com/content_link/nb7DGyGrJ1b8Rky83ThFf5eJZOp5n3tREARyL5eQmJjKmwK3gYyAvtxLrAqulshe/file
Resolving dl.dropboxusercontent.com... 45.58.70.5
Connecting to dl.dropboxusercontent.com|45.58.70.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31952 (31K) [text/plain]
Saving to: “topUsers_Apr-Jul_2014_1000-words_summaries.txt?dl=0”

100%[======================================>] 31,952      --.-K/s   in 0.001s  

2016-06-10 13:30:08 (51.9 MB/s) - “topUsers_Apr-Jul_2014_1000-words_summar

In [69]:
!mv 'topUsers_Apr-Jul_2014_1000-words_summaries.txt?dl=0' \
topUsers_Apr-Jul_2014_1000-words_summaries.txt

In [73]:
!mv topUsers_Apr-Jul_2014_1000-words_summaries.txt /home/cloudera/w261/HW4/data/

In [79]:
!wget "https://www.dropbox.com/sh/m0nxsf4vs5cyrp2/AACBUw7kflSmuQJ7jn-uBMV1a/topUsers_Apr-Jul_2014_1000-words.txt?dl=0"

--2016-06-10 13:47:06--  https://www.dropbox.com/sh/m0nxsf4vs5cyrp2/AACBUw7kflSmuQJ7jn-uBMV1a/topUsers_Apr-Jul_2014_1000-words.txt?dl=0
Resolving www.dropbox.com... 162.125.4.1
Connecting to www.dropbox.com|162.125.4.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/Mg7vlcfW3cPBctv3S7updHGnFz3pgvLQ1meWCJVqpj0xpBto4hWuldRTnOlTofJq/file [following]
--2016-06-10 13:47:07--  https://dl.dropboxusercontent.com/content_link/Mg7vlcfW3cPBctv3S7updHGnFz3pgvLQ1meWCJVqpj0xpBto4hWuldRTnOlTofJq/file
Resolving dl.dropboxusercontent.com... 108.160.173.165
Connecting to dl.dropboxusercontent.com|108.160.173.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2552940 (2.4M) [text/plain]
Saving to: “topUsers_Apr-Jul_2014_1000-words.txt?dl=0”

100%[======================================>] 2,552,940   10.1M/s   in 0.2s    

2016-06-10 13:47:08 (10.1 MB/s) - “topUsers_Apr-Jul_2014_1000-words.txt?dl=0” sav

In [262]:
!head -n 300 /home/cloudera/w261/HW4/data/topUsers_Apr-Jul_2014_1000-words.txt \
> /home/cloudera/w261/HW4/data/mini_topUsers_Apr-Jul_2014_1000-words.txt

In [48]:
%%writefile Kmeans.py
#!/home/cloudera/anaconda2/bin/python

from numpy import argmin, array, random
import numpy as np
from mrjob.job import MRJob
from mrjob.step import MRStep
from itertools import chain
import os

## function to determine the most similar cluster to a 
## data point
def MinDist(datapoint, centroid_points):
    
    ## create np array for data point vector and centroids
    datapoint = array(datapoint)
    centroid_points = array(centroid_points)
    
    ## calculate difference between the data point
    ## and each centroid and square the results
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    
    ## select centroid with smallest Euclidean distance
    ## to the data point and return it
    minidx = argmin(list(diffsq.sum(axis = 1)))
    # return the index for the closest centroid
    return minidx

## Check k-means convergence
def stop_criterion(centroid_points_old, centroid_points_new,T):
    
    ## produce array of coordinates for old and new centroids
    oldvalue = list(chain(*centroid_points_old))
    newvalue = list(chain(*centroid_points_new))
    
    ## compute the absolute difference between the new
    ## and old values for each coordinate point
    Diff = [abs(a-b) for a, b in zip(oldvalue, newvalue)]
    
    Flag = True
    
    ## Iterate through all parameters' absolute differences
    for i in Diff:
        
        ## if any of the dimensions' differences is above the convergence
        ## threshold, then fail the convergence check
        if(i>T):
            ## False flag entails failure to converge
            Flag = False
            break
    return Flag

class MRKmeans(MRJob):
    
    centroid_points=[]
    k=4    

    ## steps include a mapper_init for the loading of the
    ## centroids and a combiner to sum the points for the
    ## average reqiured to calculate new centroids in the
    ## reducer
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer
                    )
               ]

    ## load centroids info from file
    def mapper_init(self):
        print "Current path:", os.path.dirname(os.path.realpath(__file__))
        ## turn each element into a float for each line in the file
        ## we split the line by "\n" and take the first element because
        ## that is the line. we could also just do strip()
        self.centroid_points = [map(float,s.split('\n')[0].split(',')) for s in open("/home/cloudera/w261/HW4/src/Centroids.txt").readlines()]
    ## mapper input: point
    ## mapper output: key   = most similar cluster 
    ##                value = normalized data point 
    def mapper(self, _, line):
        # convert the line into an array of floats
        point = (map(float,line.split(',')))
        norm_point = map(lambda x: x/point[2], point[3:])
        # yield:
        # key: centroid closest to the point
        # value: the x- and y- coordinates of the point
        yield int(MinDist(norm_point,self.centroid_points)), (1, norm_point)

    def combiner(self, idx, inputdata):
        ## set the sum of rows to zero
        num = 0
        ## initialize aggregate for cluster to zero
        agg_points = np.array([0.0] * 1000)
        points = iter(inputdata)
        for point in points:
            num = num + point[0]
            agg_points = agg_points + point[1:]
        ## yield cluster (key), sum of cluster points
        ## processed and coordinates sum (values)
        yield idx, (num, agg_points.tolist())
        
    #Combine sum of data points locally
    def reducer(self, idx, inputdata):
        # initialize centroids list
        centroids = []
        # create an array of zeros of length k
        num = [0]*self.k 
        # loop through the number of centroids 
        # and add the coordinates 0,0
        for i in range(self.k):
            centroids.append(np.array([0.0] * 1000))
        points = iter(inputdata)
        # for each coordinate in the input data
        for point in points:
            num[idx] = num[idx] + point[0]
            centroids[idx] = centroids[idx] + point[1:]
            
        ## yield cluster (key), sum of cluster points
        ## processed and coordinates sum (values)
        yield idx, centroids[idx].tolist()
        
if __name__ == '__main__':
    MRKmeans.run()

Overwriting Kmeans.py


In [40]:
!chmod a+x Kmeans.py
!./Kmeans.py \
/home/cloudera/w261/HW4/data/mini_topUsers_Apr-Jul_2014_1000-words.txt

No configs found; falling back on auto-configuration
Creating temp directory /tmp/Kmeans.cloudera.20160611.035523.754292
Running step 1 of 1...
Current path: /home/cloudera/w261/HW4/src
Current path: /home/cloudera/w261/HW4/src
Streaming final output from /tmp/Kmeans.cloudera.20160611.035523.754292/output...
3	[300, [[13.548096544042034, 10.111101023400018, 6.467392327114453, 6.5439902312053295, 6.006209962842992, 5.731553123919869, 4.894143260074289, 3.44649173992998, 3.978246862313286, 3.824210400550781, 3.7415484833083763, 3.696380465015727, 3.1028762037348274, 2.8463508048915873, 2.5808116049572867, 2.9468664772959654, 2.878626067131466, 2.6303332774399384, 2.5592398407557346, 2.3964990045810217, 1.913594130033815, 2.261084206071635, 1.8396521820234977, 1.5694566251209912, 1.9442433796866374, 1.7672506356398556, 1.5553328598889875, 1.7463903279318742, 1.3786020004463644, 1.3563182493266908, 1.3412417632387748, 1.3026058377697995, 1.0645933745382812, 1.3540567670320536, 1.3056373526

##### ANSWER:

Here we tackle part (A), initializing four clusters to four random users in the dataset and computing K-means until convergence.

In [36]:
%reload_ext autoreload
%autoreload 2

import numpy
from Kmeans import MRKmeans, stop_criterion
mr_job = MRKmeans(args=['/home/cloudera/w261/HW4/data/mini_topUsers_Apr-Jul_2014_1000-words.txt',
                        '--file=Centroids.txt'])

###########################
# Geneate initial centroids
###########################
centroid_points = []
k = 4
init_users = numpy.random.choice(300, k)
csv = numpy.genfromtxt ('/home/cloudera/w261/HW4/data/mini_topUsers_Apr-Jul_2014_1000-words.txt',
                     delimiter=",")

for i in init_users:
    centroid_points.append(csv[i,3:])

#############################
# Finish generating centroids
#############################

## Writing init centroids to 
## disk
with open('Centroids.txt', 'w') as f:
        f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)

# Update centroids iteratively
i = 0
while(1):
    # save previous centoids to check convergency
    centroid_points_old = centroid_points[:]
    print "iteration"+str(i)+":"
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            print "CHECK THIS OUTPUT OUT\n\n\n"
            key,value =  mr_job.parse_output_line(line)
            print key, value
            centroid_points[key] = value
            
        # Update the centroids for the next iteration
        with open('Centroids.txt', 'w') as f:
            f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)
        
    print "\n"
    i = i + 1
    if(stop_criterion(centroid_points_old,centroid_points,0.01)):
        break
print "Centroids\n"
print centroid_points

iteration0:
Current path: /tmp/Kmeans.cloudera.20160611.034536.363181/job_local_dir/0/mapper/0
Current path: /tmp/Kmeans.cloudera.20160611.034536.363181/job_local_dir/0/mapper/1
CHECK THIS OUTPUT OUT



1 [[[13.548096544042032, 10.111101023400018, 6.467392327114451, 6.543990231205335, 6.006209962842992, 5.731553123919867, 4.894143260074289, 3.4464917399299813, 3.9782468623132865, 3.8242104005507813, 3.741548483308377, 3.6963804650157277, 3.1028762037348274, 2.846350804891587, 2.5808116049572876, 2.9468664772959645, 2.8786260671314645, 2.630333277439937, 2.559239840755733, 2.3964990045810213, 1.913594130033815, 2.261084206071636, 1.8396521820234972, 1.569456625120993, 1.944243379686637, 1.7672506356398567, 1.5553328598889875, 1.7463903279318744, 1.3786020004463635, 1.3563182493266899, 1.3412417632387736, 1.302605837769799, 1.064593374538281, 1.3540567670320538, 1.30563735266613, 1.2927004637788033, 1.2900538331539522, 1.0735439518435796, 1.1118544184564907, 1.106089115179468, 1.21394682

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [32]:
import numpy

###########################
# Geneate initial centroids
###########################
centroid_points = []
k = 4
init_users = numpy.random.choice(300, k)
csv = numpy.genfromtxt ('/home/cloudera/w261/HW4/data/mini_topUsers_Apr-Jul_2014_1000-words.txt',
                     delimiter=",")

for i in init_users:
    centroid_points.append(csv[i,3:])

#############################
# Finish generating centroids
#############################

## Writing init centroids to 
## disk
with open('Centroids.txt', 'w') as f:
        f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)

In [37]:
import numpy
from Kmeans import MRKmeans, stop_criterion
mr_job = MRKmeans(args=['/home/cloudera/w261/HW4/data/mini_topUsers_Apr-Jul_2014_1000-words.txt',
                        '--file=Centroids.txt'])

###########################
# Geneate initial centroids
###########################
centroid_points = []
k = 4
init_users = numpy.random.choice(1000, k)
csv = numpy.genfromtxt ('/home/cloudera/w261/HW4/data/mini_topUsers_Apr-Jul_2014_1000-words.txt',
                     delimiter=",")

for i in init_users:
    centroid_points.append(csv[i,3:])

#############################
# Finish generating centroids
#############################

## Writing init centroids to 
## disk
with open('Centroids.txt', 'w') as f:
        f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)
    # save previous centoids to check convergency
centroid_points_old = centroid_points[:]
print "iteration"+str(i)+":"
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        print "CHECK THIS OUTPUT OUT\n\n\n"
        key,value =  mr_job.parse_output_line(line)
        print key, value

iteration235:
Current path: /tmp/Kmeans.cloudera.20160611.035240.713523/job_local_dir/0/mapper/0
Current path: /tmp/Kmeans.cloudera.20160611.035240.713523/job_local_dir/0/mapper/1
CHECK THIS OUTPUT OUT



3 [[[13.548096544042032, 10.111101023400018, 6.467392327114451, 6.543990231205335, 6.006209962842992, 5.731553123919867, 4.894143260074289, 3.4464917399299813, 3.9782468623132865, 3.8242104005507813, 3.741548483308377, 3.6963804650157277, 3.1028762037348274, 2.846350804891587, 2.5808116049572876, 2.9468664772959645, 2.8786260671314645, 2.630333277439937, 2.559239840755733, 2.3964990045810213, 1.913594130033815, 2.261084206071636, 1.8396521820234972, 1.569456625120993, 1.944243379686637, 1.7672506356398567, 1.5553328598889875, 1.7463903279318744, 1.3786020004463635, 1.3563182493266899, 1.3412417632387736, 1.302605837769799, 1.064593374538281, 1.3540567670320538, 1.30563735266613, 1.2927004637788033, 1.2900538331539522, 1.0735439518435796, 1.1118544184564907, 1.106089115179468, 1.213946

In [46]:
import numpy
from Kmeans import MRKmeans, stop_criterion
mr_job = MRKmeans(args=['/home/cloudera/w261/HW4/data/mini_topUsers_Apr-Jul_2014_1000-words.txt',
                        '--file=Centroids.txt'])

###########################
# Geneate initial centroids
###########################
centroid_points = []
k = 4
init_users = numpy.random.choice(1000, k)
csv = numpy.genfromtxt ('/home/cloudera/w261/HW4/data/topUsers_Apr-Jul_2014_1000-words.txt',
                     delimiter=",")

for i in init_users:
    centroid_points.append(csv[i,3:])

#############################
# Finish generating centroids
#############################

## Writing init centroids to 
## disk
with open('Centroids.txt', 'w') as f:
        f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)